# Read Data

In [2]:
import pandas as pd

# Pick out 
def Data_compare(data,window):
    view_count = 0
    request_num = 0
    start_time = -1
    timestamp = 0
    end_time = 0
    idx = []
    for i in range(data.shape[0]):
        timestamp = int(str(data.iloc[i,0])[3:10])
        if start_time == -1:
            start_time = timestamp + (window-1) * 86400
            end_time = start_time + window * 86400
        if end_time < timestamp:
            break
        if (timestamp >= start_time and timestamp <= end_time):
            idx.append(i)
    print(idx[0],'min',idx[-1],'max')
    return idx

data = pd.read_csv("./test_20180920 _orgin")
data["Video_ID"] = pd.Categorical(data['Video_ID'], categories=data["Video_ID"].unique()).codes
idx = Data_compare(data,7)

train_data = data.iloc[:idx[0]]
test_data = data.iloc[idx[0]:idx[-1],1]
test_data.to_csv("./testing_1001_video.csv",index=0)

print(train_data.shape)
print(test_data.shape)


110415 min 248457 max
(110415, 8)
(138042,)


# Data Preprocess

### Delete repeated rows

In [3]:
train_data = train_data.drop_duplicates(subset=['Video_ID'], keep="first")
print(train_data.shape)

(57353, 8)


### Delete view_count == 0  rows


In [3]:
train_data = train_data[train_data.Video_Count != 0]
train_data["Video_ID"].to_csv("./training_1001_video.csv",index=0)

### data sorted

In [4]:
train_data = train_data.sort(['Video_Count'],ascending=False)
print(train_data)

            Req_time  Video_ID  Video_Category  Video_Count  Like_Count  \
33543   1.201806e+09     20714               1   1002205775     1487329   
55      1.201640e+09        48              23    850764845     1657651   
6633    1.201663e+09      4687              10    534786167     2751681   
6634    1.201663e+09      4688              23    299358415     1203195   
38555   1.201822e+09     23501              10    290252671      527616   
3359    1.201653e+09      2527              26    279060475        2977   
5545    1.201660e+09      3982              10    260780126     1079685   
65      1.201640e+09        58              10    215102526     1069962   
4895    1.201658e+09      3551              10    204147380      407396   
10770   1.201679e+09      7221              10    201162163      858872   
84013   1.202019e+09     45826              10    190476499     1231718   
4049    1.201655e+09      3003              10    175273975      385168   
6653    1.201663e+09     

C:\Users\wmnet\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


### Categorize Labels (y)

In [5]:
#labels = ['0','class2','class3','class4','popular']
labels = [0,1,2,3,4]
train_data['labels'] = pd.qcut(train_data.Video_Count, 5, labels=labels)
print(train_data['labels'])

33543     4
55        4
6633      4
6634      4
38555     4
3359      4
5545      4
65        4
4895      4
10770     4
84013     4
4049      4
6653      4
34775     4
89821     4
24476     4
197       4
4121      4
41161     4
34090     4
17661     4
18851     4
18527     4
17956     4
21388     4
3920      4
19020     4
11789     4
12249     4
22386     4
         ..
29926     0
33305     0
33365     0
74955     0
33418     0
56585     0
103810    0
45746     0
20457     0
89797     0
38914     0
24949     0
60304     0
33347     0
12161     0
33211     0
10127     0
55231     0
8020      0
33361     0
49789     0
97745     0
54928     0
102408    0
24575     0
36056     0
48507     0
103323    0
33430     0
33428     0
Name: labels, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]


In [6]:
train_data.shape

(57350, 9)

In [7]:
train_data = train_data.sort_index(axis=0)
train_data.to_csv("./training_1001_labeled.csv",index=0)

## HT Model

In [12]:
# Very Fast Decision Tree i.e. Hoeffding Tree, described in
# "Mining High-Speed Data Streams" (Domingos & Hulten, 2000)
#
# this program contains 2 classes: Vfdt, VfdtNode
# test in command line window: python3 Vfdt.py
# changed to CART: gini index
#
# Jamie
# 02/06/2018
# ver 0.03

import time
from itertools import combinations
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# VFDT node class
class VfdtNode:
    def __init__(self, possible_split_features):
        """
        nijk: statistics of feature i, value j, class
        :list possible_split_features: features
        """
        self.parent = None
        self.left_child = None
        self.right_child = None
        self.split_feature = None
        self.split_value = None  # both continuous and discrete value
        self.new_examples_seen = 0
        self.total_examples_seen = 0
        self.class_frequency = {}
        self.nijk = {f: {} for f in possible_split_features}
        self.possible_split_features = possible_split_features

    def add_children(self, split_feature, split_value, left, right):
        self.split_feature = split_feature
        self.split_value = split_value
        self.left_child = left
        self.right_child = right
        left.parent = self
        right.parent = self

        self.nijk.clear()  # reset stats
        if isinstance(split_value, list):
            left_value = split_value[0]
            right_value = split_value[1]
            # discrete split value list's length = 1, stop splitting
            if len(left_value) <= 1:
                new_features = [None if f == split_feature else f for f in left.possible_split_features]
                left.possible_split_features = new_features
            if len(right_value) <= 1:
                new_features = [None if f == split_feature else f for f in right.possible_split_features]
                right.possible_split_features = new_features

    def is_leaf(self):
        return self.left_child is None and self.right_child is None

    # recursively trace down the tree to distribute data examples to corresponding leaves
    def sort_example(self, x):
        if self.is_leaf():
            return self
        else:
            index = self.possible_split_features.index(self.split_feature)
            value = x[index]
            split_value = self.split_value

            if isinstance(split_value, list):  # discrete value
                if value in split_value[0]:
                    return self.left_child.sort_example(x)
                else:
                    return self.right_child.sort_example(x)
            else:  # continuous value
                if value <= split_value:
                    return self.left_child.sort_example(x)
                else:
                    return self.right_child.sort_example(x)

    # the most frequent class
    def most_frequent(self):
        try:
            prediction = max(self.class_frequency, key=self.class_frequency.get)
        except ValueError:
            # if self.class_frequency dict is empty, go back to parent
            class_frequency = self.parent.class_frequency
            prediction = max(class_frequency, key=class_frequency.get)
        return prediction

    # update leaf stats in order to calculate infomation gain
    def update_stats(self, x, y):
        feats = self.possible_split_features
        nijk = self.nijk
        iterator = [f for f in feats if f is not None]
        for i in iterator:
            value = x[feats.index(i)]
            if value not in nijk[i]:
                nijk[i][value] = {y: 1}
            else:
                try:
                    nijk[i][value][y] += 1
                except KeyError:
                    nijk[i][value][y] = 1

        self.total_examples_seen += 1
        self.new_examples_seen += 1
        class_frequency = self.class_frequency
        try:
            class_frequency[y] += 1
        except KeyError:
            class_frequency[y] = 1

    def check_not_splitting(self):
        # compute gini index for not splitting
        X0 = 1
        class_frequency = self.class_frequency
        n = sum(class_frequency.values())
        for j, k in class_frequency.items():
            X0 -= (k/n)**2
        return X0

    # use Hoeffding tree model to test node split, return the split feature
    def attempt_split(self, delta, nmin, tau):
        if self.new_examples_seen < nmin:
            return None
        class_frequency = self.class_frequency
        if len(class_frequency) == 1:
            return None

        self.new_examples_seen = 0  # reset
        nijk = self.nijk
        min = 1
        second_min = 1
        Xa = ''
        split_value = None
        for feature in self.possible_split_features:
            if feature is not None:
                njk = nijk[feature]
                gini, value = self.gini(njk, class_frequency)
                if gini < min:
                    min = gini
                    Xa = feature
                    split_value = value
                elif min < gini < second_min:
                    second_min = gini

        epsilon = self.hoeffding_bound(delta)
        g_X0 = self.check_not_splitting()
        if min < g_X0:
            # print(second_min - min, epsilon)
            if second_min - min > epsilon:
                # print('1 node split')
                return [Xa, split_value]
            elif second_min - min < epsilon < tau:
                # print('2 node split')
                return [Xa, split_value]
            else:
                return None
        return None

    def hoeffding_bound(self, delta):
        n = self.total_examples_seen
        R = np.log(len(self.class_frequency))
        return np.sqrt(R * R * np.log(1/delta) / (2 * n))

    def gini(self, njk, class_frequency):
        # gini(D) = 1 - Sum(pi^2)
        # gini(D, F=f) = |D1|/|D|*gini(D1) + |D2|/|D|*gini(D2)

        D = self.total_examples_seen
        m1 = 1  # minimum gini
        # m2 = 1  # second minimum gini
        Xa_value = None
        feature_values = list(njk.keys())  # list() is essential
        if not isinstance(feature_values[0], str):  # numeric  feature values
            sort = np.array(sorted(feature_values))
            split = (sort[0:-1] + sort[1:])/2   # vectorized computation, like in R

            D1_class_frequency = {j: 0 for j in class_frequency.keys()}
            for index in range(len(split)):
                nk = njk[sort[index]]
                for j in nk:
                    D1_class_frequency[j] += nk[j]
                D1 = sum(D1_class_frequency.values())
                D2 = D - D1
                g_d1 = 1
                g_d2 = 1

                D2_class_frequency = {}
                for key, value in class_frequency.items():
                    if key in D1_class_frequency:
                        D2_class_frequency[key] = value - D1_class_frequency[key]
                    else:
                        D2_class_frequency[key] = value

                for key, v in D1_class_frequency.items():
                    g_d1 -= (v/D1)**2
                for key, v in D2_class_frequency.items():
                    g_d2 -= (v/D2)**2
                g = g_d1*D1/D + g_d2*D2/D
                if g < m1:
                    m1 = g
                    Xa_value = split[index]
                # elif m1 < g < m2:
                    # m2 = g
            return [m1, Xa_value]

        else:  # discrete feature_values
            length = len(njk)
            if length > 10:  # too many discrete feature values, estimate
                for j, k in njk.items():
                    D1 = sum(k.values())
                    D2 = D - D1
                    g_d1 = 1
                    g_d2 = 1

                    D2_class_frequency = {}
                    for key, value in class_frequency.items():
                        if key in k:
                            D2_class_frequency[key] = value - k[key]
                        else:
                            D2_class_frequency[key] = value
                    for key, v in k.items():
                        g_d1 -= (v/D1)**2

                    if D2 != 0:
                        for key, v in D2_class_frequency.items():
                            g_d2 -= (v/D2)**2
                    g = g_d1*D1/D + g_d2*D2/D
                    if g < m1:
                        m1 = g
                        Xa_value = j
                    # elif m1 < g < m2:
                        # m2 = g
                right = list(np.setdiff1d(feature_values, Xa_value))

            else:  # fewer discrete feature values, get combinations
                comb = self.select_combinations(feature_values)
                for i in comb:
                    left = list(i)
                    D1_class_frequency = {key: 0 for key in class_frequency.keys()}
                    D2_class_frequency = {key: 0 for key in class_frequency.keys()}
                    for j,k in njk.items():
                        for key, value in class_frequency.items():
                            if j in left:
                                if key in k:
                                    D1_class_frequency[key] += k[key]
                            else:
                                if key in k:
                                    D2_class_frequency[key] += k[key]
                    g_d1 = 1
                    g_d2 = 1
                    D1 = sum(D1_class_frequency.values())
                    D2 = D - D1
                    for key, v in D1_class_frequency.items():
                        g_d1 -= (v/D1)**2
                    for key, v in D2_class_frequency.items():
                        g_d2 -= (v/D2)**2
                    g = g_d1*D1/D + g_d2*D2/D
                    if g < m1:
                        m1 = g
                        Xa_value = left
                    # elif m1 < g < m2:
                        # m2 = g
                right = list(np.setdiff1d(feature_values, Xa_value))
            return [m1, [Xa_value, right]]

    # divide values into two groups, return the combination of left groups
    def select_combinations(self, feature_values):
        combination = []
        e = len(feature_values)
        if e % 2 == 0:
            end = int(e/2)
            for i in range(1, end+1):
                if i == end:
                    cmb = list(combinations(feature_values, i))
                    enough = int(len(cmb)/2)
                    combination.extend(cmb[:enough])
                else:
                    combination.extend(combinations(feature_values, i))
        else:
            end = int((e-1)/2)
            for i in range(1, end+1):
                combination.extend(combinations(feature_values, i))

        return combination


# very fast decision tree class, i.e. hoeffding tree
class Vfdt:
    def __init__(self, features, delta=0.01, nmin=100, tau=0.1):
        """
        :features: list of data features
        :delta: used to compute hoeffding bound, error rate
        :nmin: to limit the G computations
        :tau: to deal with ties
        """
        self.features = features
        self.delta = delta
        self.nmin = nmin
        self.tau = tau
        self.root = VfdtNode(features)
        self.n_examples_processed = 0

    # update the tree by adding training example
    def update(self, x, y):
        self.n_examples_processed += 1
        node = self.root.sort_example(x)
        node.update_stats(x, y)

        result = node.attempt_split(self.delta, self.nmin, self.tau)
        if result is not None:
            feature = result[0]
            value = result[1]
            self.node_split(node, feature, value)

    # split node, produce children
    def node_split(self, node, split_feature, split_value):
        features = node.possible_split_features
        # print('node_split')
        left = VfdtNode(features)
        right = VfdtNode(features)
        node.add_children(split_feature, split_value, left, right)

    # predict test example's classification
    def predict(self, x_test):
        prediction = []
        if isinstance(x_test, np.ndarray) or isinstance(x_test, list):
            for x in x_test:
                leaf = self.root.sort_example(x)
                prediction.append(leaf.most_frequent())
            return prediction
        else:
            leaf = self.root.sort_example(x_test)
            return leaf.most_frequent()

    def print_tree(self, node):
        if node.is_leaf():
            print('Leaf')
        else:
            print(node.split_feature)
            self.print_tree(node.left_child)
            self.print_tree(node.right_child)


def calc_metrics(y_test, y_pred, row_name):
    accuracy = accuracy_score(y_test, y_pred)
    metrics = list(precision_recall_fscore_support(y_test, y_pred, average='weighted',
                                                   labels=np.unique(y_pred)))
    metrics = pd.DataFrame({'accuracy': accuracy, 'precision': metrics[0],'recall': metrics[1],
                            'f1': metrics[2]}, index=[row_name])
    return metrics


def test_run():
    start_time = time.time()

    df = pd.read_csv('./training_1001_labeled.csv', header=None, sep=',', skiprows=1)
    df_test = pd.read_csv('./testing_1001_video.csv',header=None,sep=',', skiprows=1)
    # df = pd.read_csv('./dataset/default_of_credit_card_clients.csv', skiprows=1, header=0)
    
    #df = df.drop(df.columns[0], axis=1)
    
    #df = df.sample(frac=1).reset_index(drop=True)  # shuffle data rows
    title = list(df.columns.values)
    features = title[:-1]
    features = [0,1,2,3,4,5,6]
    rows = df.shape[0]


    #n_training = int(df.shape[0] * 0.4)
    #training_data = df.iloc[:n_training]
    #n_cache = int(df.shape[0] * 0.6)
    #n_training = n_training - n_cache
    
    # date head
    array = df.head(df.shape[0]).values
    array = np.delete(array,1, axis=1)
    x_train__ = array[:,:-1]
    y_train__ = array[:,-1]
    set1 = array[:int(df.shape[0] / 3), :]
    set2 = array[int(df.shape[0] / 3):int(df.shape[0] / 3 * 2), :]
    set3 = array[int(df.shape[0] / 3 * 2):, :]
    
    # cache head
    #cache_boxes = df.tail(n_cache).values
    #cache_boxes = np.delete(cache_boxes,1, axis=1)
    #x_cache = cache_boxes[:,:-1]
    #y_cache = cache_boxes[:,-1]

    # to simulate continuous training, modify the tree for each training set
    examples = [set1, set2, set3]

    # test set is different from training set
    #n_test = int(df.shape[0] * 0.3)
    #test_set = df.tail(n_test).values
    #x_test = test_set[:, :-1]
    #y_test = test_set[:, -1]

    # Heoffding bound (epsilon) parameter delta: with 1 - delta probability
    # the true mean is at least r_bar - epsilon
    # Efdt parameter nmin: test split if new sample size > nmin
    # feature_values: unique values in every feature
    # tie breaking: when difference is so small, split when diff_g < epsilon < tau

    tree = Vfdt(features, delta=0.01, nmin=100, tau=0.5)

    print('Total data size: ', rows)
    #print('Training size: ', n_training)
    #print('Cache set size: ', n_cache)
    n = 0
    
    
    for training_set in examples:
        n += len(training_set)
        x_train = training_set[:, :-1]
        y_train = training_set[:, -1]
        for x, y in zip(x_train, y_train):
            tree.update(x, y)  # fit data
        y_pred = tree.predict(x_train__)
        y_array = np.array(y_pred)
        idx = np.where(y_array == 4)[0]
        idx_1 = np.where(y_array == 3)[0]
        print('Training set:', n, end=', ')
        print(y_train__.shape)
        print(len(y_pred))
        print('ACCURACY: %.4f' % accuracy_score(y_train__, y_pred))
    #df_test = pd.read_csv('./testing_1001_video.csv', header=None)
    df_video = pd.read_csv('./training_1001_video.csv', header=None)
    total_size = df_test.shape[0]
    
    #df_video = df_video.tail(n_cache).values
    order = [4,3,2,1,0]
    #cach size range {500,5000,50000}
    for length in range(100,100+1,500):
        cache_cotainer = []
        hit = 0
        while(len(cache_cotainer) < length):
            for i in range(0,4+1):
                idx = np.where(y_array == order[i])[0]
                for ii in idx:
                    if len(cache_cotainer) < length:
                        cache_cotainer.append(df_video.iloc[ii,0])
                    else:
                        break
        for item in df_test.iloc[:,0]:
            if item in cache_cotainer:
                hit += 1
        print("Cach Size %5f ---Hit Rate: %f" % (length,hit/total_size*100))
        
        
            
    # tree.print_tree(tree.root)
    print("--- Running time: %.6f seconds ---" % (time.time() - start_time))

In [13]:
import numpy as np
test_run()

Total data size:  57350
Training set: 19116, (57350,)
57350
ACCURACY: 0.9976
Training set: 38233, (57350,)
57350
ACCURACY: 0.9994
Training set: 57350, (57350,)
57350
ACCURACY: 0.9995
Cach Size 100.000000 ---Hit Rate: 0.704863
--- Running time: 2.939139 seconds ---


In [238]:
test_run()

C:\Users\wmnet\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2827: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


[0, 1, 2, 3, 4, 5, 6, 7, 8]
Total data size:  90003
Training size:  36001
Test set size:  54001
Training set: 12000, ACCURACY: 0.5247
Training set: 24000, ACCURACY: 0.5524
Training set: 36001, ACCURACY: 0.5639
--- Running time: 5.666027 seconds ---
